
# Importing libraries 


In [1]:
from datetime import datetime

## Step 0: Preview your large file

In [2]:
%%time

from IPython.display import Video

# videofile = '/kaggle/input/google-gemini-long-context-dataset/Vijay 69.mp4'
videofile = '/kaggle/input/google-gemini-long-context-dataset/Goosebumps S01E01-E02 The Haunted Mask.mp4' 


CPU times: user 10 µs, sys: 3 µs, total: 13 µs
Wall time: 16.2 µs


In [3]:

# Video(videofile, embed=True, width=640, height=480)


## Step 1: Import Python Packages

In [4]:
import os
import time
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

print(datetime.now())

2024-11-05 19:01:23.582927


## Step 2: Authenticate with Google Generative AI

Use of Gemini will require an API key. Please visit [Google AI Studio](https://ai.google.dev/)  to generate your key. 

Next, you should attach that key to your Kaggle Notebook as a [Kaggle User Secret](https://www.kaggle.com/discussions/product-feedback/114053). 

These steps are illustrated in the following screenshots:

Attaching User Secrets: 
 - https://i.imgur.com/GjuRLCA.png
 - https://i.imgur.com/IrSXAtw.png
 
For details about pricing see https://ai.google.dev/pricing.

In [5]:

user_secrets = UserSecretsClient()

# ai_studio_token = user_secrets.get_secret("ai_studio_token")
ai_studio_token = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key=ai_studio_token)

print(datetime.now())

2024-11-05 19:01:23.749993


## Step 3: Define helper functions

In [6]:

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()


print(datetime.now())


2024-11-05 19:01:23.786777


## Step 4: Load the Gemini 1.5 model

In [7]:

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

print(datetime.now())


2024-11-05 19:01:23.821229


## Step 5: Upload your large file to Gemini 1.5

In [8]:
%%time

files = [
  # upload_to_gemini(nepali_video, mime_type="video/webm"),
    upload_to_gemini(videofile, mime_type="video/mp4"),  # Changed from video/webm to video/mp4
]

wait_for_files_active(files)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
      ],
    }
  ]
)

print(datetime.now())


# Upload time is 02 min for 255MB / 42 Min MP4 video


Uploaded file 'Goosebumps S01E01-E02 The Haunted Mask.mp4' as: https://generativelanguage.googleapis.com/v1beta/files/lbc1l9iu8npk
Waiting for file processing...
......all files ready

2024-11-05 19:01:58.570393
CPU times: user 585 ms, sys: 721 ms, total: 1.31 s
Wall time: 34.7 s


## Step 6: Ask Gemini 1.5 questions about your large file

In [9]:
%%time

# response = chat_session.send_message("Please give me the English language transcript for this video.")
# print(response.text)


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.39 µs


In [10]:
%%time 

response = chat_session.send_message("What is the video about?.")
print(response.text)

The video is a Goosebumps episode titled "The Haunted Mask". In this episode, Carly Beth puts on a haunted mask which sticks to her face, and the kids start to believe she's a real monster.  

CPU times: user 23.5 ms, sys: 13.6 ms, total: 37.2 ms
Wall time: 17.3 s


In [11]:

response = chat_session.send_message("Describe the main characters in the video?")
print(response.text)


Here are the main characters in the video and their roles in the story:

* **Carly Beth:** The main character, a young girl who is easily scared. She buys a mask from the Novelty Shop, but it sticks to her face and the kids start to think she's a real monster.
* **Sabrina:** Carly Beth's friend. She is more skeptical than Carly Beth about the mask, but ultimately gets scared too.
* **Chuck:**  A boy who enjoys scaring Carly Beth. He becomes part of the fun as the other kids start to think Carly Beth is a monster.
* **Steve:** Chuck's friend, and also enjoys the pranks on Carly Beth.
* **The Tall Thin Man:** The owner of the Novelty Shop. He is a sinister man who knows the masks have a supernatural power. 



In [12]:

# response = chat_session.send_message("Do the characters in the video resemble any popular actors?")
# print(response.text)


In [13]:
response = chat_session.send_message("Give me a hindi summary of the video?")
print(response.text)

यह वीडियो गुज़बंप्स सीरीज़ का एक एपिसोड है, जिसका नाम "द हॉन्टेड मास्क" है। इस एपिसोड में, कार्ली बेथ नाम की एक छोटी लड़की नॉवेल्टी शॉप से एक मास्क खरीदती है। लेकिन, यह मास्क उसके चेहरे पर चिपक जाता है और उसे उतार नहीं पाती।  इसके बाद, उसके दोस्त चक और स्टीव और बाकी बच्चे यह सोचने लगते हैं कि कार्ली बेथ एक असली राक्षस बन गई है।  शॉप के मालिक, लंबे और पतले आदमी को इन मास्कों की जादुई ताकत के बारे में पता है। 



In [14]:

response = chat_session.send_message("If you had to breakdown the video into segments (like paragraphs but for video). What segments would it be? Briefly describe the segments. And also give the time stamps of the same.")
print(response.text)


Here's a breakdown of the video into segments, with brief descriptions and timestamps:

**Segment 1: 0:00- 0:47**
- Introduction of R.L. Stine, the author of Goosebumps and a brief preview of the episode.

**Segment 2: 0:47- 2:11**
- Carly Beth and Sabrina walk past the Novelty Shop, and Sabrina notices the spooky masks inside. They decide to get pumpkins and then go home.

**Segment 3: 2:11- 3:59**
- Carly Beth comes home and shows her mom the clay bust she made in class, which looks like her.  She mentions the boys scaring her earlier. 

**Segment 4: 3:59- 5:52**
- Carly Beth goes back to the Novelty Shop, the Tall Thin Man mentions his masks are not for sale, but Carly Beth insists on buying one, so he gives it to her for free.

**Segment 5: 5:52- 7:55**
-  Carly Beth returns home and tries on the mask, which sticks to her face. She freaks out and the boys appear.

**Segment 6: 7:55- 10:47**
-  The Tall Thin Man explains to Carly Beth the masks are not masks, but are real faces, and

In [15]:

response = chat_session.send_message("If you had to create a trailer of the movie, without giving away the plot. What segments from the movie would you use, advise with timestamps.")
print(response.text)


Here's how I'd create a Goosebumps trailer for "The Haunted Mask" without giving away the plot:

**Opening Shot (0:00-0:05):** A shot of Carly Beth's face, but only a glimpse of the mask peeking out from the side, with spooky music playing.

**Segment 1 (0:05- 0:15):**  Cut to shots of Sabrina and Carly Beth looking into the window of the Novelty Shop, quickly flashing the most sinister-looking masks, with a slightly eerie music score building tension.

**Segment 2 (0:15-0:25):**  A shot of the Tall Thin Man, staring directly at the camera with a menacing smile. Music ramps up with a slightly unsettling tone.

**Segment 3 (0:25-0:30):**  Cut to a shot of Carly Beth wearing a costume, but from the back, so the mask isn't shown yet. We hear her laugh, which suddenly turns into a strained, panicked gasp.

**Segment 4 (0:30-0:40):**  A fast-paced montage of scary Halloween decorations, pumpkins, and costumes. The music intensifies and becomes more chaotic.

**Segment 5 (0:40-0:50):** A sho

In [16]:
print(response.usage_metadata)

prompt_token_count: 765277
candidates_token_count: 462
total_token_count: 765739



Credit:
 - Adapted from https://aistudio.google.com/app/prompts/video-qa